In [33]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Nov  2 10:41:30 2018
This program will retrieve lyrics from Genius.com
@author: ayeshamendoza
"""
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '69cf6ae94ce54c329634f9ced2c8a5fb'
client_secret = '1d6c065683bd455c992b91b873c52d6b'

##Query artist to get track IDs

def get_Spotify_trackID(title, artist):
    ''' this function send query request to Spotify by title and artist name
        and will return corresponding the track ID'''
    
    urilist = []
    error = 0
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    sp.trace=False
    search_query = title + ' ' + artist
    
    try:
        result = sp.search(search_query)
        #find a song that matches the title and artist from the query result
        for i in result['tracks']['items']:
            if (i['artists'][0]['name'] == artist) and (i['name'] == title):
                #print(i['uri'])
                urilist.append(i['uri'])
                return (urilist, error)
                break
            #else:
            #try:
                #take song returned by search
                #print(result['tracks']['items'][0]['uri'])
            #except:
             #   print('cannot find URI')
    except:
        error = 999
        print('get_Spotify_trackID error processing : ', title, artist)
    
    
                
    return (urilist, error)
    
### Getting audio data:

def getFeatures(uri):
    '''this function will send query request to Spotify for audio analysis for the specified uri'''
    
    features = []
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    sp.trace = False
    error = 0
    try:
        features = sp.audio_features(uri)
    except:
        error = 888
        print('getFeatures error processing : ' ,uri)
    
    return (features, error)

def getAudioAnalysis(uri):
    '''this function will send query request to Spotify for audio analysis for the specified uri'''
    
    features = []
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    sp.trace = False
    error = 0
    try:
        features = sp.audio_analysis(uri)
    except:
        error = 888
        print('getFeatures error processing : ' ,uri)
    
    return (features, error)





In [54]:
def processAudioDict(audioAnalysis):
   
    audioDict = {}
    
    avg_pitch = 0
    sum_pitch_mean = 0
    sum_pitch_median = []
    song_pitch_mean = 0
    song_pitch_med = 0

    sum_timbre_mean = 0
    sum_timbre_median = []
    song_timbre_mean = 0
    song_timbre_med = 0

    seg_len = len(audioAnalysis['segments'])
    audioDict['bar_len'] = len(audioAnalysis['bars'])
    audioDict['beat_len'] = len(audioAnalysis['beats'])
    audioDict['seg_len'] = len(audioAnalysis['segments'])
    
    for i in range(seg_len):
#for i in range(5):
        pitches = np.array(audioAnalysis['segments'][i]['pitches'])
        timbre = np.array(audioAnalysis['segments'][i]['timbre'])
        mean_pitch = np.mean(pitches)
        med_pitch = np.median(pitches)
        #print(mean_pitch, med_pitch)
        sum_pitch_mean += mean_pitch
        sum_pitch_median.append(med_pitch)
    
        mean_timbre = np.mean(timbre)
        med_timbre = np.median(timbre)
        sum_timbre_mean += mean_timbre
        sum_timbre_median.append(med_timbre)
    
    song_pitch_mean = sum_pitch_mean / seg_len
    song_timbre_mean = sum_timbre_mean / seg_len
    song_pitch_med = np.median(np.array(sum_pitch_median))
    song_timbre_med = np.median(np.array(sum_timbre_median))

    audioDict['pitch_mean'] = song_pitch_mean
    audioDict['pitch_med'] = song_pitch_med
    audioDict['timbre_mean'] = song_timbre_mean
    audioDict['timbre_med'] = song_timbre_med
    
    return audioDict
   

In [22]:
import pandas as pd

In [23]:
tSwiftpop = pd.read_csv('../data/msongs/out/TSwiftSongs_after2014.csv',names=['artist_name', 'title'])

In [24]:
tSwiftpop

,artist_name,title
0,Taylor Swift,Blank Space
1,Taylor Swift,Style
2,Taylor Swift,Out Of The Woods
3,Taylor Swift,All You Had to To Was Stay
4,Taylor Swift,Shake It Off
5,Taylor Swift,I Wish You Would
6,Taylor Swift,Bad Blood
7,Taylor Swift,Wildest Dreams
8,Taylor Swift,How You Get The Girl
9,Taylor Swift,This Love


In [25]:
tSwiftpop['genre'] = 'pop'

In [26]:
import pandas as pd
tSwiftCountry = pd.read_pickle('../data/msongs/out/tSwiftSongs.p')
#tSwiftpop = pd.read_csv('../data/msongs/out/TSwiftSongs_after2014',columns=[['title']])


tSwiftCountry


,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRUWKDV128F92DE274,A Perfectly Good Heart,SOFDEVD12AB017FC40,Taylor Swift,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,220.13342,0.889749,0.872447,2007,3662149,-1,0
1,TRUVVME12903CBA350,Change,SORRBVQ12A58A7AA33,Fearless Karaoke,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,280.03220,0.889749,0.872447,2008,6445662,-1,0
2,TRLCTGJ128F92DE26B,Cold As You,SOVAUWH12AB01857EA,Taylor Swift,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,238.99383,0.889749,0.922412,2006,3662140,-1,0
3,TRPKWRN128F9326B71,Cold As You,SOVAUWH12AB01857EA,Taylor Swift Karaoke,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,250.04363,0.889749,0.872447,2006,7004200,-1,0
4,TRIGYJQ128F9322C76,Come In With The Rain,SOPYYND12AB01804FC,Fearless,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,238.07955,0.889749,0.922412,2009,6987530,-1,0
5,TRUXYDP128F93234BD,Crazier,SOYGKHG12AB01804C8,Hannah Montana The Movie,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,192.73098,0.889749,0.922412,2009,4593374,-1,0
6,TRAEJFS128F92DE220,Fearless,SOSKJYK12D0219A62D,Fearless,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,243.35628,0.834496,0.880880,2008,4243045,-1,0
7,TRVZQUC128F92EC15D,Fearless,SOOAJAS12CF5169FD0,Fearless,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,242.33751,0.834477,0.922412,2008,4309419,-1,0
8,TRKVVVK12903CB592C,Fifteen,SODUDUN12AB018CCAB,My Songs 2010,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,242.05016,0.889749,0.922412,2008,8377828,-1,0
9,TRHCDSY128F931692E,Forever & Always,SOMPTCI12AB017C416,Fearless,ARS54I31187FB46721,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,224.99220,0.889749,0.922412,2008,4526572,-1,0


In [27]:
tSwiftCountry =tSwiftCountry[['artist_name', 'title']].copy()

In [28]:
tSwiftCountry['genre'] = 'country'

In [29]:
tSwiftDF = pd.DataFrame(data=None, columns=tSwiftCountry.columns)
tSwiftDF = tSwiftpop.append(tSwiftCountry, ignore_index=True)

In [30]:
tSwiftDF.drop_duplicates(subset=['artist_name', 'title'], inplace=True)

In [31]:
tSwiftDF.reset_index(drop=True, inplace=True)
tSwiftDF

,artist_name,title,genre
0,Taylor Swift,Blank Space,pop
1,Taylor Swift,Style,pop
2,Taylor Swift,Out Of The Woods,pop
3,Taylor Swift,All You Had to To Was Stay,pop
4,Taylor Swift,Shake It Off,pop
5,Taylor Swift,I Wish You Would,pop
6,Taylor Swift,Bad Blood,pop
7,Taylor Swift,Wildest Dreams,pop
8,Taylor Swift,How You Get The Girl,pop
9,Taylor Swift,This Love,pop


In [55]:
for i, row in enumerate(tSwiftDF.itertuples(), start=0):
    
    print('processing row {}'.format(i))
    # get track id:
    urilist, err = get_Spotify_trackID(row.title, row.artist_name)
    if err > 0:
        print('...breaking')
        tSwiftDF.to_pickle('tSwiftDF.p')
        #break
    else: 
        if len(urilist) == 1:
            print('getting Features....')
            uri = urilist[0].split(':')[2]
            songFeatures, err = getFeatures(uri)
            if err > 0:
                print('breaking....')
                tSwiftDF.to_pickle('tSwiftDF.p')
                #break
            else:
                tSwiftDF.set_value(i,'spotifyURI',uri)
                tSwiftDF.set_value(i,'songFeatures',songFeatures)
                audioAnalysis, err = getAudioAnalysis(uri)
                
                
                audioDict = processAudioDict(audioAnalysis)
                
                print(audioDict)
                
                if err > 0:
                    print('breaking....')
                    tSwiftDF.to_pickle('tSwiftDF.p')
                else:
                    tSwiftDF.set_value(i,'bar_len',audioDict['bar_len'])
                    tSwiftDF.set_value(i,'beat_len',audioDict['beat_len'])
                    tSwiftDF.set_value(i,'seg_len',audioDict['seg_len'])
                    tSwiftDF.set_value(i,'pitch_mean',audioDict['pitch_mean'])
                    tSwiftDF.set_value(i,'pitch_med',audioDict['pitch_med'])
                    tSwiftDF.set_value(i,'timbre_mean',audioDict['timbre_mean'])
                    tSwiftDF.set_value(i,'timbre_med',audioDict['timbre_med'])
                #row.songFeatures = songFeatures
                #print(uri)
    

processing row 0
getting Features....
{'bar_len': 92, 'beat_len': 368, 'seg_len': 856, 'pitch_mean': 0.3548530957943923, 'pitch_med': 0.21975, 'timbre_mean': 11.008000194704039, 'timbre_med': 6.527}
processing row 1
getting Features....
{'bar_len': 89, 'beat_len': 360, 'seg_len': 931, 'pitch_mean': 0.3579622269960616, 'pitch_med': 0.252, 'timbre_mean': 3.6539904224847817, 'timbre_med': -0.7014999999999999}
processing row 2
getting Features....
{'bar_len': 88, 'beat_len': 353, 'seg_len': 954, 'pitch_mean': 0.4212158455625438, 'pitch_med': 0.3315, 'timbre_mean': 5.662791928721176, 'timbre_med': 2.1987499999999995}
processing row 3
processing row 4
getting Features....
{'bar_len': 144, 'beat_len': 575, 'seg_len': 872, 'pitch_mean': 0.3962704510703366, 'pitch_med': 0.303, 'timbre_mean': 12.340521120030596, 'timbre_med': 6.3332500000000005}
processing row 5
getting Features....
{'bar_len': 99, 'beat_len': 400, 'seg_len': 891, 'pitch_mean': 0.4083016273849609, 'pitch_med': 0.31, 'timbre_mean

getting Features....
{'bar_len': 156, 'beat_len': 628, 'seg_len': 727, 'pitch_mean': 0.3174524300779458, 'pitch_med': 0.175, 'timbre_mean': 5.742218133883541, 'timbre_med': 1.173}
processing row 47
getting Features....
{'bar_len': 255, 'beat_len': 1024, 'seg_len': 954, 'pitch_mean': 0.2812615303983231, 'pitch_med': 0.14025, 'timbre_mean': 2.210931953179593, 'timbre_med': 0.6}
processing row 48
getting Features....
{'bar_len': 88, 'beat_len': 355, 'seg_len': 685, 'pitch_mean': 0.25635012165450116, 'pitch_med': 0.106, 'timbre_mean': 0.8017298053527973, 'timbre_med': -1.4125}
processing row 49
getting Features....
{'bar_len': 123, 'beat_len': 492, 'seg_len': 738, 'pitch_mean': 0.3152021228545621, 'pitch_med': 0.1785, 'timbre_mean': 6.441154810298101, 'timbre_med': 3.0805}
processing row 50
getting Features....
{'bar_len': 145, 'beat_len': 576, 'seg_len': 685, 'pitch_mean': 0.2635227493917273, 'pitch_med': 0.11599999999999999, 'timbre_mean': 8.99028880778589, 'timbre_med': 4.098}


In [56]:
tSwiftDF

,artist_name,title,genre,spotifyURI,songFeatures,bar_len,beat_len,seg_len,pitch_mean,pitch_med,timbre_mean,timbre_med
0,Taylor Swift,Blank Space,pop,1p80LdxRV74UKvL8gnD7ky,"[{'danceability': 0.752, 'energy': 0.678, 'key...",92.0,368.0,856.0,0.354853,0.21975,11.008000,6.52700
1,Taylor Swift,Style,pop,4lIxdJw6W3Fg4vUIYCB0S5,"[{'danceability': 0.598, 'energy': 0.786, 'key...",89.0,360.0,931.0,0.357962,0.25200,3.653990,-0.70150
2,Taylor Swift,Out Of The Woods,pop,5OndtwLGA9O6XHFcGm2H7r,"[{'danceability': 0.552, 'energy': 0.841, 'key...",88.0,353.0,954.0,0.421216,0.33150,5.662792,2.19875
3,Taylor Swift,All You Had to To Was Stay,pop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Taylor Swift,Shake It Off,pop,5xTtaWoae3wi06K5WfVUUH,"[{'danceability': 0.648, 'energy': 0.785, 'key...",144.0,575.0,872.0,0.396270,0.30300,12.340521,6.33325
5,Taylor Swift,I Wish You Would,pop,3wmN11qXGL0HyPojvEE1D9,"[{'danceability': 0.649, 'energy': 0.898, 'key...",99.0,400.0,891.0,0.408302,0.31000,9.537195,4.02950
6,Taylor Swift,Bad Blood,pop,273dCMFseLcVsoSWx59IoE,"[{'danceability': 0.652, 'energy': 0.802, 'key...",146.0,585.0,802.0,0.409555,0.28700,11.074240,6.07500
7,Taylor Swift,Wildest Dreams,pop,59HjlYCeBsxdI0fcm3zglw,"[{'danceability': 0.553, 'energy': 0.664, 'key...",125.0,504.0,768.0,0.373953,0.25400,5.754274,3.27850
8,Taylor Swift,How You Get The Girl,pop,6wfugRLamFsTRbPcCpNnP7,"[{'danceability': 0.765, 'energy': 0.656, 'key...",122.0,487.0,849.0,0.371223,0.26000,9.722701,4.99450
9,Taylor Swift,This Love,pop,1kTPQnabROVkW9bUXdCGrB,"[{'danceability': 0.481, 'energy': 0.435, 'key...",147.0,588.0,783.0,0.289739,0.12850,-0.419021,-1.52300


In [50]:
tSwiftDF.loc[12,'title'] = '...Ready For It'

In [ ]:
    if (x < dflen) & (x % 100 == 0):
        print('processing row {}'.format(i), row.title, row.artist_name)
        songsDF.to_pickle('spotifyGenre_1122_bkp.p')
    x += 1
    
print('Get Spotify Features for pop songs ended')
songsDF.to_pickle('SpotifyAudioFeatures_done.p')

print(songsDF[['artist_name','title','spotifyURI','songFeatures','audioAnalysis']][:10])